### Fit a regression model

In [1]:
import ssl
import urllib.request
import pandas as pd
import os

# save .dat file
dat_url = 'https://www2.stat.duke.edu/~pdh10/FCBS/Exercises/azdiabetes.dat'
dat_file_path = 'azdiabetes.dat'

ssl_context = ssl._create_unverified_context()

# Download the .dat file with the unverified SSL context
with urllib.request.urlopen(dat_url, context=ssl_context) as response:
    with open(dat_file_path, 'wb') as out_file:
        out_file.write(response.read())

# Define the directory to save the .csv file
save_directory = '/Users/cherylshi/UROP/UROP'
os.makedirs(save_directory, exist_ok=True)

# Define the .csv file path
csv_file_name = 'azdiabetes.csv'
csv_file_path = os.path.join(save_directory, csv_file_name)

# Read the .dat file into a pandas DataFrame
df = pd.read_csv(dat_file_path, delimiter='\s+')

# Save the DataFrame as a .csv file
df.to_csv(csv_file_path, index=False)
os.remove(dat_file_path)

In [2]:
import scipy.stats as stats
import statsmodels.api as sm

# Exclude the 'diabetes' variable
X = df.drop(columns=['diabetes', 'glu'])
y = df['glu']

In [3]:
# Define prior parameters
n = 532
g = n
nu0 = 2
sigma02 = 1

# number of independent samples
s = 1000

In [4]:
import numpy as np
import math

#sample from the distributions
factor = X @ np.linalg.inv(X.T @ X) @ np.array(X.T)
SSRg = (y.T) @ (np.identity(n) - (g/(g+1)) * factor) @ y
shape = (nu0 + n) / 2
scale = (nu0 * sigma02 + SSRg) / 2
gamma_samples = stats.gamma.rvs(a=shape, scale=scale, size=s)

sigma = 1 / math.sqrt(sum(gamma_samples)/s)

mean = (g/(g+1)) * (np.linalg.inv(X.T @ X) @ np.array(X.T) @ y)
cov = (g/(g+1)) * (sum(gamma_samples)/s) * np.linalg.inv(X.T @ X)

mvnormal_samples = np.random.multivariate_normal(mean, cov, s)

beta = sum(mvnormal_samples) / s

print(sigma, beta)

0.00012527833220860922 [-2.46757362e+00 -1.42638485e-01 -2.40201724e-03  1.60316179e+00
 -1.46214359e+01  2.56902724e+00]


### Model Selection by Gibbs sampling

In [5]:
import numpy as np
from scipy.special import gammaln

# log probability
def lp_y_X(y, X):
    
    n, p = X.shape
    
    if p == 0:
        Hg = 0
        sigma02 = np.mean(y**2)
    else:
        Hg = (g/(g+1)) * factor
        sigma02 = 1
    
    SSRg = y.T @ (np.identity(n) - Hg) @ y 
    log_prob = -0.5 * (n * np.log(np.pi) + p * np.log(1 + g) 
                       + (nu0 + n) * np.log(nu0 * sigma02 + SSRg) 
                       - nu0 * np.log(nu0 * sigma02)) + gammaln((nu0 + n) / 2) - gammaln(nu0 / 2)
    return log_prob

# Starting values and MCMC setup
S = 10000
Z = np.full((S, X.shape[1]), np.nan)
z = np.ones(X.shape[1], dtype=int)

# Initial log probability
lp_y_c = lp_y_X(y, X.iloc[:, z == 1])

# Gibbs sampler
for s in range(S):
    for j in np.random.permutation(X.shape[1]):
        zp = z.copy()
        zp[j] = 1 - zp[j]
        lp_y_p = lp_y_X(y, X.iloc[:, zp == 1])
        r = (lp_y_p - lp_y_c) * (-1) ** (zp[j] == 0)
        z[j] = np.random.binomial(1, 1 / (1 + np.exp(-r)))
        if z[j] == zp[j]:
            lp_y_c = lp_y_p
    Z[s, :] = z


### Metropolis-Hastings

In [6]:
X1 = df[['npreg', 'bp', 'bmi', 'ped', 'age']]

# centering and scaling
means = X1.mean()
stds = X1.std()
X_scaled = (X1 - means) / stds
X_scaled['intercept'] = 1
X_scaled = X_scaled[['intercept', 'npreg', 'bp', 'bmi', 'ped', 'age']].values


y1 = df['diabetes'].replace({'Yes': 1, 'No': 0})

/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_90218/3048451862.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y1 = df['diabetes'].replace({'Yes': 1, 'No': 0})


In [7]:
import numpy as np
from scipy.stats import multivariate_normal
from numpy.random import uniform

In [8]:
# Log likelihood function
def log_L(x):
    p = np.exp(x) / (1 + np.exp(x))
    return sum(y1) * np.log(p) + (len(y1)-sum(y1)) * np.log(1-p)

In [9]:
# Initial values
n_iter = 10000
beta = np.ones(6)
gamma = np.ones(6)
p_beta = multivariate_normal.pdf(beta, np.zeros(6),
                                 np.diag([16, 4, 4, 4, 4, 4]))

# Store samples
beta_samples = np.zeros((n_iter, 6))
gamma_samples = np.zeros((n_iter, 6))

# proporsal variance can be approximated by \sigma^2(X^{T} X)^{-1}, which is close to the posterior variance of \beta
var_prop = np.var(np.log(y1+1/2)) * np.linalg.inv(X_scaled.T @ X_scaled)

# Metropolis-Hastings sampling
for i in range(n_iter):
    # Sample beta
    beta_prop = np.random.multivariate_normal(beta, var_prop)
    J_prop_beta = multivariate_normal.pdf(beta_prop, beta, var_prop)
    p_prop_beta = multivariate_normal.pdf(beta_prop, np.zeros(6),
                                          np.diag([16, 4, 4, 4, 4, 4]))
    J_beta = multivariate_normal.pdf(beta, beta_prop, var_prop)
    L_prop_beta = sum(log_L(sum(beta_prop * gamma * X_scaled)))
    L_beta = sum(log_L(sum(beta * gamma * X_scaled)))

    r_log_beta = L_prop_beta + np.log(p_prop_beta) - L_beta - np.log(p_beta) + np.log(J_beta) - np.log(J_prop_beta)

    if np.log(uniform()) < min(0, r_log_beta):
        beta = beta_prop
        p_beta = p_prop_beta

    beta_samples[i, :] = beta
    print(r_log_beta)


    # Sample gamma
    for r in range(1,6):
        gamma_prop = gamma.copy()
        gamma_prop[r] = 1 - gamma_prop[r]

        L_prop_gamma = sum(log_L(sum(beta * gamma_prop * X_scaled)))
        L_gamma = log_L(sum(beta * gamma * X_scaled))
        r_log_gamma = sum(L_prop_gamma - L_gamma)

        if np.log(uniform()) < min(0, r_log_gamma):
            gamma = gamma_prop
    
    gamma_samples[i, :] = gamma


/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_90218/929354117.py:4: RuntimeWarning: divide by zero encountered in log
  return sum(y1) * np.log(p) + (len(y1)-sum(y1)) * np.log(1-p)
/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_90218/2201829057.py:26: RuntimeWarning: invalid value encountered in scalar subtract
  r_log_beta = L_prop_beta + np.log(p_prop_beta) - L_beta - np.log(p_beta) + np.log(J_beta) - np.log(J_prop_beta)
/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_90218/2201829057.py:43: RuntimeWarning: invalid value encountered in subtract
  r_log_gamma = sum(L_prop_gamma - L_gamma)


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_90218/929354117.py:3: RuntimeWarning: overflow encountered in exp
  p = np.exp(x) / (1 + np.exp(x))
/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_90218/929354117.py:3: RuntimeWarning: invalid value encountered in divide
  p = np.exp(x) / (1 + np.exp(x))


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [10]:
gamma_samples

array([[1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])

In [11]:
beta = np.ones(6)
gamma = np.ones(6)
sum(beta * gamma * X_scaled)

array([ 5.32000000e+02,  1.29896094e-14, -1.41747725e-13,  1.05415676e-13,
        8.69304628e-14, -5.97299987e-14])

In [12]:
beta_samples

array([[ 9.83062598e-01,  9.91892568e-01,  9.60761781e-01,
         9.76664339e-01,  9.82038226e-01,  1.01937533e+00],
       [ 9.61364739e-01,  1.03119843e+00,  9.95120764e-01,
         9.71688337e-01,  9.78692267e-01,  9.89604666e-01],
       [ 9.95197147e-01,  1.03900664e+00,  9.87137603e-01,
         9.80728939e-01,  9.61512025e-01,  9.83594152e-01],
       ...,
       [-1.04378703e-03,  1.29805444e+00,  3.21603475e+00,
         7.24031963e-01,  6.78240534e-01,  2.97153804e+00],
       [-1.04378703e-03,  1.29805444e+00,  3.21603475e+00,
         7.24031963e-01,  6.78240534e-01,  2.97153804e+00],
       [-1.04378703e-03,  1.29805444e+00,  3.21603475e+00,
         7.24031963e-01,  6.78240534e-01,  2.97153804e+00]])

In [14]:
r_log_beta

np.float64(-2759.0853244901095)

In [24]:
sum(L_prop_beta)

np.float64(-inf)

In [20]:
beta_prop * gamma * X_scaled

array([[ 9.94511471e-01,  4.50744545e-01, -2.77168831e-01,
        -3.97733445e-01, -3.98035379e-01, -7.01216393e-01],
       [ 9.94511471e-01,  1.05859270e+00, -1.19041412e-01,
        -1.15173735e+00, -9.73751856e-01,  2.15350115e+00],
       [ 9.94511471e-01,  4.50744545e-01,  8.29723102e-01,
         4.30192407e-01, -9.93801683e-01,  3.11747895e-01],
       ...,
       [ 9.94511471e-01,  1.97036494e+00,  3.55340845e-01,
         1.44509077e-03, -9.50837767e-01,  2.89020245e+00],
       [ 9.94511471e-01,  4.50744545e-01,  3.90860068e-02,
        -9.89109054e-01, -7.38882447e-01, -1.48690418e-01],
       [ 9.94511471e-01, -7.64951770e-01, -1.19041412e-01,
        -3.68164665e-01, -5.38384172e-01, -7.93304056e-01]])

In [23]:
beta_prop * gamma

array([0.99451147, 1.00660744, 0.97329431, 1.01732999, 0.98687042,
       0.9910091 ])

In [22]:
gamma

array([1., 1., 1., 1., 1., 1.])

In [6]:
z

array([0, 0, 1, 0, 0, 0])